实现一下多输入通道互相关运算。 简而言之，我们所做的就是对每个通道执行互相关操作，然后将结果相加

In [1]:
import torch
from d2l import torch as d2l

def corr2d_multi_in(X, K):
    # 先遍历“X”和“K”的第0个维度（通道维度），再把它们加在一起
    return sum(d2l.corr2d(x, k) for x, k in zip(X, K))

验证互相关运算的输出

In [2]:
X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
               [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])

corr2d_multi_in(X, K)

tensor([[ 56.,  72.],
        [104., 120.]])

计算多个通道的输出的互相关函数

 ```
 torch.Size([3, 2, 2, 2])
            ↓  ↓  ↓  ↓
            |  |  |  └── 卷积核宽度 = 2
            |  |  └───── 卷积核高度 = 2  
            |  └──────── 输入通道数 = 2 (必须匹配X的通道数)
            └─────────── 输出通道数 = 3 (因为stack了3个(K, K + 1, K + 2))

In [4]:
def corr2d_multi_in_out(X, K):
    # 迭代“K”的第0个维度，每次都对输入“X”执行互相关运算。
    # 最后将所有结果都叠加在一起
    return torch.stack([corr2d_multi_in(X, k) for k in K], 0)

K = torch.stack((K, K + 1, K + 2), 0)
K.shape

torch.Size([3, 2, 2, 2])

我们对输入张量X与卷积核张量K执行互相关运算。现在的输出包含3个通道，第一个通道的结果与先前输入张量X和多输入单输出通道的结果一致。

之所以第一个向量和之前的结果一样，是因为其实我们K,K+1和K+2其实是在对同一个内容做卷积，只是这次我们的卷积核换成了3个。在现实的CNN中，不同的卷积核就像是专门的特征检测器。

In [5]:
corr2d_multi_in_out(X, K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

1×1卷积

当以像素为基础应用时，1×1卷积层相当于全连接层

In [6]:
def corr2d_multi_in_out_1x1(X, K):
    #c_i表示"channels input"（输入通道数）；h表示"height"（高度）；w表示"width"（宽度）；c_o表示"channels output"（输出通道数）
    c_i, h, w = X.shape
    c_o = K.shape[0]
    X = X.reshape((c_i, h * w))
    K = K.reshape((c_o, c_i))
    # 全连接层中的矩阵乘法
    Y = torch.matmul(K, X)
    return Y.reshape((c_o, h, w))

X = torch.normal(0, 1, (3, 3, 3))
K = torch.normal(0, 1, (2, 3, 1, 1))

Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)
assert float(torch.abs(Y1 - Y2).sum()) < 1e-6    